In [1]:
import numpy as np
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [2]:
dataset = pd.read_csv('tweets-total-csv-3-class-unbalanced.csv', encoding='ISO-8859-1', delimiter=';')

In [3]:
tweets = dataset['tweet'].values
classes = dataset['OPINIAO'].values

In [4]:
def Preprocessamento(instancia):
    #remove links, pontos, virgulas,ponto e virgulas dos tweets
    #coloca tudo em minusculo
    fraseslimpas = []
    for palavras in instancia:
        palavras = re.sub(r"http\S+", "", palavras).lower().replace(',','').replace('.','').replace(';','').replace('-','').replace(':','').replace('?','')
        fraseslimpas.append(palavras)
    return (fraseslimpas)

In [5]:
def RemoveStopWords(instancia):
    #remove stopwords
    fraseslimpas = []
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    stopwords.remove('não')
    for palavras in instancia:
        semstop = [p for p in palavras.split() if p not in stopwords]
        fraseslimpas.append((str(semstop)))
    return (fraseslimpas)

In [6]:
def SteamingSemStopWords(instancia):
    fraseslimpas = []
    stemmer = nltk.stem.RSLPStemmer()
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    stopwords.remove('não')
    for palavras in instancia:
        semstop = [str(stemmer.stem(p)) for p in palavras.split() if p not in stopwords]
        fraseslimpas.append((str(semstop)))
    return (fraseslimpas)

In [7]:
tweetsprocessados = Preprocessamento(tweets)
tweetsprocessadossteaming = SteamingSemStopWords(tweetsprocessados)

In [8]:
print(tweets[1])
print(tweetsprocessados[1])
print(tweetsprocessadossteaming[1])

O #BancoDoBrasil já me deixou uma mensagem de parabéns. O 1º.
o #bancodobrasil já me deixou uma mensagem de parabéns o 1º
['#bancodobrasil', 'deix', 'mens', 'parabém', '1º']


In [9]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweetsprocessadossteaming)
modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=50)

print('Porcentagem de acerto: %f ' % (metrics.accuracy_score(classes, resultados)*100))

Porcentagem de acerto: 72.101090 


In [11]:
opiniao=['Positivo', 'Negativo', 'Neutro']
print(metrics.classification_report(classes,resultados,opiniao))

              precision    recall  f1-score   support

    Positivo       0.25      0.19      0.22       166
    Negativo       0.77      0.92      0.84      1299
      Neutro       0.66      0.41      0.50       553

    accuracy                           0.72      2018
   macro avg       0.56      0.51      0.52      2018
weighted avg       0.70      0.72      0.70      2018



In [13]:
from nltk.metrics import ConfusionMatrix
esperado = []
previsto = []
for result in resultados:
    previsto.append(result)
for classe in classes:
    esperado.append(classe)    

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

         |    N         P |
         |    e         o |
         |    g    N    s |
         |    a    e    i |
         |    t    u    t |
         |    i    t    i |
         |    v    r    v |
         |    o    o    o |
---------+----------------+
Negativo |<1198>  67   34 |
  Neutro |  266 <225>  62 |
Positivo |   86   48  <32>|
---------+----------------+
(row = reference; col = test)



In [12]:
dataset2 = pd.read_csv('Dados3.csv', encoding='ISO-8859-1', delimiter=';')

In [40]:
dataset2['Text'][1]

'@senadorhumberto Duas merdas: PT e Folha de São Paulo.  \n\nEm breve não existirá nenhum .'

In [41]:
tweets = dataset2['Text'].values
classes = dataset2['Opniao'].values

In [42]:
tweetsprocessados = Preprocessamento(tweets)
tweetsprocessadossteaming = SteamingSemStopWords(tweetsprocessados)

In [43]:
print(tweets[1])
print(tweetsprocessados[1])
print(tweetsprocessadossteaming[1])

@senadorhumberto Duas merdas: PT e Folha de São Paulo.  

Em breve não existirá nenhum .
@senadorhumberto duas merdas pt e folha de são paulo  

em breve não existirá nenhum 
['@senadorhumbert', 'dua', 'merd', 'pt', 'folh', 'paul', 'brev', 'não', 'existir', 'nenhum']


In [44]:
freq_tweets = vectorizer.transform(tweetsprocessadossteaming)

In [45]:
opniao = modelo.predict(freq_tweets)

In [46]:
print('Porcentagem de acerto: %f ' % (metrics.accuracy_score(classes, opniao)*100))

Porcentagem de acerto: 57.142857 


In [47]:
tipo=['Positivo', 'Negativo', 'Neutro']
print(metrics.classification_report(classes,opniao,tipo))

              precision    recall  f1-score   support

    Positivo       0.33      0.25      0.29         4
    Negativo       0.88      0.50      0.64        14
      Neutro       0.47      0.80      0.59        10

    accuracy                           0.57        28
   macro avg       0.56      0.52      0.50        28
weighted avg       0.65      0.57      0.57        28



In [48]:
from nltk.metrics import ConfusionMatrix
esperado = []
previsto = []
for opniao in opniao:
    previsto.append(opniao)
for classe in classes:
    esperado.append(classe)    

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

         | N   P |
         | e   o |
         | g N s |
         | a e i |
         | t u t |
         | i t i |
         | v r v |
         | o o o |
---------+-------+
Negativo |<7>6 1 |
  Neutro | 1<8>1 |
Positivo | . 3<1>|
---------+-------+
(row = reference; col = test)



In [49]:
def BuscaOpiniao(frase):
    tweetsprocessados = Preprocessamento(frase)
    tweetsprocessadossteaming = SteamingSemStopWords(tweetsprocessados)
    freq_tweets = vectorizer.transform(tweetsprocessadossteaming)
    opniao = modelo.predict(freq_tweets)
    prob = modelo.predict_proba(freq_tweets).max()
    return opniao,prob

In [50]:
opiniao = BuscaOpiniao([dataset2['Text'][0]])
print(opiniao[0][0])

Neutro


In [51]:
dataset3 = pd.read_csv('Dados2.csv', encoding='ISO-8859-1', delimiter=';')
datasetClassificado = pd.DataFrame(columns=['Text','Opiniao','Probabilidade'])

In [52]:
for text in dataset3['Text']:
    global opiniao
    opiniao = BuscaOpiniao([text])
    datasetClassificado = datasetClassificado.append({'Text':text,'Opiniao':opiniao[0][0],'Probabilidade':opiniao[1]}, ignore_index=True)

In [53]:
datasetClassificado.to_excel('DadosClassificadosB.xlsx')